# Convert given array as a sparse connected fcn

![](img/example1.png)

![](img/example2.png)

## Connection also represents its acitvation function
- 0 : not connected
- 1 : linear 
- 2 : ReLU
- 3 : Sigmoid
- and so on..

**this can be changed**

# References
- [Concatenate layer output with additional input data](https://discuss.pytorch.org/t/concatenate-layer-output-with-additional-input-data/20462)

In [84]:
import numpy as np
import torch
import torch.nn as nn

# class MatrixForWANN()

In [2]:
class MatrixForWANN() : 
    def __init__(self, mat, in_dim, out_dim) : 
        
        # get when intiliazed
        self.mat = mat
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        # calculate
        self.num_hidden_nodes = self.mat.shape[1]-self.in_dim
        self.hidden_dim = self.get_hidden_dim()
        
    def get_hidden_dim(self):
        in_dim = self.in_dim
        out_dim = self.out_dim
        mat_mask = self.mat
        hidden_dim_list = []
        start_col_idx = 0 
        finish_col_idx = in_dim-1

        while(True):

            if finish_col_idx >= mat_mask.shape[1]:
                break

            for i in range(start_col_idx, len(mat_mask)): 
                if (mat_mask[i,start_col_idx:(finish_col_idx + 1)].sum() == 0):
                    hidden_dim = i - sum(hidden_dim_list)
                    hidden_dim_list += [hidden_dim] 
                    start_col_idx = finish_col_idx + 1
                    finish_col_idx += i 
                    break
        return hidden_dim_list

In [3]:
mat = np.array([[0,2,0,0,2,0,0,0,0,0],
                [2,0,2,0,0,0,0,0,0,0],
                [0,2,0,2,0,0,0,0,0,0],
                [0,0,0,0,0,1,1,0,0,0],
                [0,0,0,0,0,0,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,3],
                [0,0,0,0,0,0,0,0,3,0]])
in_dim = 5
out_dim = 2
mat_wann = MatrixForWANN(mat, in_dim, out_dim)

In [4]:
# first position represents row : FROM
mat[0]

array([0, 2, 0, 0, 2, 0, 0, 0, 0, 0])

In [5]:
# second position represents column : TO
mat[:, 0]

array([0, 2, 0, 0, 0, 0, 0])

In [6]:
# get destinations of input layer
mat[:mat_wann.num_hidden_nodes, :in_dim]

array([[0, 2, 0, 0, 2],
       [2, 0, 2, 0, 0],
       [0, 2, 0, 2, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [7]:
# number of nodes for each hidden layer
mat_wann.hidden_dim

[3, 2]

# class WANNFCN

In [9]:
mat_wann.num_hidden_nodes

5

In [10]:
# get hidden nodes which is connected from input layer
# 1 : connected, 0 : not connected
# positional index : node index for given hidden node
def hiddens_from_input(mat, num_hidden_nodes, in_dim) : 
    hidden_nodes_connected_from_input = []
    for i in range(num_hidden_nodes) : 
        hidden_nodes_connected_from_input.append(1 if mat[i, :in_dim].sum() != 0 else 0)
    return hidden_nodes_connected_from_input
hiddens_from_input(mat, mat_wann.num_hidden_nodes, in_dim)

# hidden nodes 중에서, input이랑 연결된 애들의 index

[1, 1, 1, 0, 0]

In [32]:
# get index of input node which is connected to the given hidden node
idx_hidden_node = 0
mat[idx_hidden_node, :in_dim]

# 주어진 hiddden node에 대해서, 얘랑 연결되는 input node

array([0, 2, 0, 0, 2])

In [31]:
def output_from_hiddens(mat, num_hidden_nodes, out_dim) : 
    output_nodes_connected_from_hidden = []
    for i in range(mat.shape[0]-out_dim, mat.shape[1]) : 
        output_nodes_connected_from_hidden.append(1 if mat[-out_dim:, i].sum() != 0 else 0)
    return output_nodes_connected_from_hidden
output_from_hiddens(mat, mat_wann.num_hidden_nodes, out_dim)

[0, 0, 0, 1, 1]

In [34]:
# get index of hidden node which is connected to the given output node
idx_output_node = 0
mat[mat_wann.num_hidden_nodes+idx_output_node, in_dim:]

array([0, 0, 0, 0, 3])

In [35]:
# activations = [None, None, self.relu, self.sigmoid]
def wrap_activation(x, idx_activation, activations) : 
    if idx_activation == 1 :
        return x
    elif idx_activation == 2 :
        return activations[2](x)
    elif idx_activation == 3 :
        return activations[3](x)
    else : 
        return False

In [36]:
mat_wann.hidden_dim

[3, 2]

In [38]:
mat_wann.num_hidden_nodes

5

In [ ]:
Linear(28, 1)[27]

In [ ]:
class WANNFCN

- Linear를 마지막ㅇ 넣으면 weight bias가 또 생길수 있음
- Activation function 씌울때 Linear를 먼저 만들고, activation을 씌워야..
- 그러니까 Linear를 정의할게 아니라 concat으로만 주는게 ...?

In [83]:
class WANNFCN(nn.Module) : 
    def __init__(self, mat_wann) : 
        super(WANNFCN, self).__init__()
        self.mat = mat_wann.mat
        self.in_dim = mat_wann.in_dim
        self.out_dim = mat_wann.out_dim
        self.num_hidden_nodes = mat_wann.num_hidden_nodes
        self.hidden_dim = mat_wann.hidden_dim
        
        self.relu = nn.ReLU() # 2
        self.sigmoid = nn.Sigmoid() # 3
        self.activations = [None, None, self.relu, self.sigmoid]
        
    def forward(self, x) : # forward determines connections
        nodes = {}
        #node_counts = 0 # number of nodes connected 
        for idx_hidden_layer, num_hidden_nodes in enumerate(self.hidden_dim) :

            # 1. First layer
            if idx_hidden_layer == 0 :
                # index of hidden nodes in first layer
                indices_hidden = list(range(0, num_hidden_nodes))
                print(indices_hidden)
                # check connection for nodes in first layer
                # idx_hidden : index of hidden node (0~last index of first layer)
                # connection : 1-connected, 0-not
                for idx_hidden, connection in enumerate(hiddens_from_input(self.mat, self.num_hidden_nodes, self.in_dim)[:num_hidden_nodes]) : 
                    if connection != 0 : # connected
                        # array below contains info. about connection from input and its activation
                        # e.g.) self.mat[idx_hidden, :self.in_dim] : array([0, 2, 0, 0, 2])
                        # idx_input : index of input node
                        # activation_type : 0-not connected, 1,2,3..-connected with given activation function
                        count_connection = 0 
                        input_node = None
                        for idx_input, activation_type in enumerate(self.mat[idx_hidden, :self.in_dim]) :
                            # connected first input node
                            if activation_type != 0 and count_connection == 0  :
                                # x[idx_for_batch, idx_for_position]
                                input_node = wrap_activation(x[:, idx_input], activation_type, self.activations).reshape(-1, 1)
                                count_connection += 1
                            # connected other input node
                            elif activation_type != 0 and count_connection != 0 : 
                                input_node = torch.cat([input_node,
                                                       wrap_activation(x[:, idx_input], activation_type, self.activations).reshape(-1, 1)
                                                       ], dim=1)
                                count_connection += 1

                        # connect corresponding input node to nodes['hidden_(node number)']
                        input_node = input_node.view(-1, count_connection)
                        nodes['hidden_%d'%idx_hidden] = nn.Linear(count_connection, 1)(input_node)
                
                #node_counts += num_hidden_nodes
            print(nodes)
            # 2. Hidden Layers
            
            '''
            난 내일 이 코드를 다시 열어봤을때 내 코드를 이해하거나 리팩토링할 자신이 없다
            흑흑....
            '''

            '''
            # 3. Last Layer
            if idx_hidden_layer == len(self.hidden_dim)-1 : 
                
                indices_output = list(range(0, self.out_dim)) # in row
                indices_output = [x+self.num_hidden_nodes for x in indices_output]
                print('indices_output', indices_output)
                
                for idx_output in indices_output : # output node에 대해 도는 loop
                    print('outputidx(row) : %s, output:%s' %(idx_output, self.mat[idx_output, self.in_dim:]))
                    count_connection = 0
                    hidden_node = None
                    
                    for idx_hidden, connection in enumerate(self.mat[idx_output, self.in_dim:]) : # hidden node에 대해 도는 loop
                        print('hidden idx(column) : %s, connection : %s' %(idx_hidden+self.in_dim, connection))
                        
                        if connection != 0 : # connected
                            print('activation_type %s count_connection %s connection %s' % (activation_type, count_connection, connection))
                            if connection != 0 and count_connection == 0 :
                                print('case 1 start')
                                hidden_node = wrap_activation(nodes['hidden_%d'%(idx_hidden)], activation_type, self.activations).reshape(-1, 1)
                                count_connection += 1
                                print('case 1 end')
                            elif connection != 0 and count_connection != 0 :
                                print('case 2 start')
                                hidden_node = torch.cat([hidden_node, wrap_activation(nodes['hidden_%d'%(idx_hidden)], 
                                                                                      activation_type,
                                                                                      self.activations).reshape(-1, 1)], dim=1)
                                count_connection += 1
                                print('case 2 end')
                    hidden_node = hidden_node.view(-1, count_connection)
                    nodes['output_%d'%(idx_output-self.num_hidden_nodes)] = nn.Linear(count_connection, 1)(hidden_node)
            '''
            
    
                    
                
               
                

        print(nodes)
        '''
        nodes라는 dictionary 안에 아래와 같이 저장됨
        'hidden_1' : 해당 노드
        'hidden_2' : 해당 노드
        ...
        'output_1' : 해당 output 노드
        'output_2' : 해당 output 노드
        '''
        output_1 = activation(hidden_5*weight + bias) + activaiton(hidden_6*weight + bias)

        
    
    
model = WANNFCN(mat_wann)

numpy_input = np.array([[1,2,3,4,5],
                        [6,7,8,9,10],
                        [11,12,13,14,15]])
numpy_input = torch.from_numpy(numpy_input).float()

model(numpy_input)

[0, 1, 2]
{'hidden_0': tensor([[0.2935],
        [0.5902],
        [0.8869]], grad_fn=<AddmmBackward>), 'hidden_1': tensor([[0.8985],
        [2.2537],
        [3.6088]], grad_fn=<AddmmBackward>), 'hidden_2': tensor([[-1.1512],
        [-4.5000],
        [-7.8488]], grad_fn=<AddmmBackward>)}
{'hidden_0': tensor([[0.2935],
        [0.5902],
        [0.8869]], grad_fn=<AddmmBackward>), 'hidden_1': tensor([[0.8985],
        [2.2537],
        [3.6088]], grad_fn=<AddmmBackward>), 'hidden_2': tensor([[-1.1512],
        [-4.5000],
        [-7.8488]], grad_fn=<AddmmBackward>)}
{'hidden_0': tensor([[0.2935],
        [0.5902],
        [0.8869]], grad_fn=<AddmmBackward>), 'hidden_1': tensor([[0.8985],
        [2.2537],
        [3.6088]], grad_fn=<AddmmBackward>), 'hidden_2': tensor([[-1.1512],
        [-4.5000],
        [-7.8488]], grad_fn=<AddmmBackward>)}


In [ ]:
'''
    def forward(self, x) : # forward determines connections
        # 
        nodes={}   
        # 1. get indices of first nodes connected to first hidden layer
        # e.g.) hiddens_from_input : [1, 1, 1, 0, 0], 1: connected, 0: not connected
        # idx_hidden : index of hidden node
        # connection : 1-connected, 0-not
        for idx_hidden, connection  in enumerate(hiddens_from_input(self.mat, self.in_dim)) :
            if connection != 0 : # connected
                # array below contains info. about connection from input and its activation
                # e.g.) self.mat[idx_hidden, :self.in_dim] : array([0, 2, 0, 0, 2])
                # idx_input : index of input node
                # activation_type : 0-not connected, 1,2,3..-connected with given activation function
                print(self.mat[idx_hidden, :self.in_dim])
                count_connection = 0 
                input_node = None
                for idx_input, activation_type in enumerate(self.mat[idx_hidden, :self.in_dim]) :
                    # for debugging
                    print(idx_input, count_connection, activation_type, input_node)
                    # connected first input node
                    if activation_type != 0 and count_connection == 0  :
                        # x[idx_for_batch, idx_for_position]
                        input_node = wrap_activation(x[:, idx_input], activation_type, self.activations).reshape(-1, 1)
                        count_connection += 1
                    # connected other input node
                    elif activation_type != 0 and count_connection != 0 : 
                        input_node = torch.cat([input_node,
                                               wrap_activation(x[:, idx_input], activation_type, self.activations).reshape(-1, 1)
                                               ], dim=1)
                        count_connection += 1
                    
                # connect corresponding input node to nodes['hidden_(node number)']
                input_node = input_node.view(-1, count_connection)
                print(input_node, nn.Linear(count_connection, 1), count_connection)
                nodes['hidden_%d'%idx_hidden] = nn.Linear(count_connection, 1)(input_node)


        return nodes
''''''